In [ ]:
# March 2025
# Script to preprocess data from wos and ce database
# Violeta Berdejo-Espinola

In [1]:
# %pip install fasttext-wheel

import pandas as pd
from pathlib import Path

In [2]:
# function to normalize text

def normalize_text(text):
    text = text.replace('<i>','')
    text = text.replace('</i>','')
    text = text.replace('*','')
    return text.lower().strip().capitalize()

# read wos data

In [3]:
# wos data

path = Path(f'../data/from_wos/')

# find broken files in wos folder

for file in path.glob("*.xls"):
    try:
        df = pd.read_excel(file)
        print(f"Successfully read {file}")
    except Exception as e:
        print(f"Error reading {file}: {e}")

Successfully read ../data/from_wos/id_278.xls
Successfully read ../data/from_wos/id_254_3.xls
Successfully read ../data/from_wos/id_244.xls
Successfully read ../data/from_wos/id_76_3.xls
Successfully read ../data/from_wos/id_235_2.xls
Successfully read ../data/from_wos/id_76_11.xls
Successfully read ../data/from_wos/id_213_1.xls
Successfully read ../data/from_wos/id_94_4.xls
Successfully read ../data/from_wos/id_31_1.xls
Successfully read ../data/from_wos/id_17_2.xls
Successfully read ../data/from_wos/id_134_2.xls
Successfully read ../data/from_wos/id_109_2.xls
Successfully read ../data/from_wos/id_126.xls
Successfully read ../data/from_wos/id_101_11.xls
Successfully read ../data/from_wos/id_49_1.xls
Successfully read ../data/from_wos/id_28_1.xls
Successfully read ../data/from_wos/id_195_2.xls
Successfully read ../data/from_wos/id_269_2.xls
Successfully read ../data/from_wos/id_90_1.xls
Successfully read ../data/from_wos/id_252_4.xls
Successfully read ../data/from_wos/id_64_10.xls
Succ

In [33]:
# read all xls files in the folder into a list of dfs
path = Path(f'../data/from_wos/')
dfs = [pd.read_excel(file) for file in path.glob("*.xls")]

# concatenate dfs into one master df

df_concat = pd.concat(dfs, ignore_index=True)
print(f"unique journal names {df_concat['Source Title'].nunique()}")

# rename columns

df_master = df_concat.rename(columns={
    'Article Title': 'title',
    'Abstract': 'abstract',
    'Source Title': 'journal',
    'Authors': 'authors',
    'Publication Year': 'year'
})

# normalize journal name

df_master["journal"] = df_master["journal"].apply(normalize_text)

# normalize article title
 
df_master['title'] = df_master['title'].apply(normalize_text)

# subset metadata

df_repo = df_master.loc[:,['title', 'abstract', 'journal', 'year', 'authors']]
print(f'NAs in repo:\n{df_repo.isna().sum()}')
print(f'articles in repo raw: {len(df_repo)}')

# create df of nas and duplicates

df_repo_nas = df_repo[df_repo['abstract'].isna() | df_repo['authors'].isna()]
df_repo_dupes = df_repo[df_repo.duplicated(subset=['title','authors'], keep=False)]

# remove unwanted articles 

df_repo = df_repo[~df_repo["title"].str.contains("In Memoriam|Editorial|Correction|Notes|Untitled|Retracted")]
print(f'articles in repo: {len(df_repo)} -> editorials, in memoriam removed')

# remove nas and duplicates

df_repo = df_repo.dropna(subset='authors', how='any').dropna(subset='abstract', how='any').drop_duplicates(subset=['title','abstract'], keep='first')

print(f'articles in repo: {len(df_repo)} -> nas, duplicates removed')
print(f"unique journals in repo {df_repo['journal'].nunique()}")

# create journal list

repo_journals = df_repo['journal'].unique()

unique journal names 286
NAs in repo:
title           0
abstract    77831
journal         0
year            0
authors        69
dtype: int64
articles in repo raw: 525433
articles in repo: 523393 -> editorials, in memoriam removed
articles in repo: 440346 -> nas, duplicates removed
unique journals in repo 274


# detect language of titles in repo 


In [34]:
from fasttext.FastText import _FastText

model_path = 'lid.176.ftz'
model = _FastText(model_path=model_path)

# function to detect languages

def get_lang(text: str) -> str:
    lang, _ = model.predict(text)
    lang = lang[0].removeprefix('__label__')
    # conf = conf[0]

    return lang

# detect languages

df_repo["language"] = df_repo["title"].apply(get_lang)

print(df_repo["language"].value_counts())

# filter english only

df_repo = df_repo[df_repo['language'] == 'en']
df_repo = df_repo.reset_index()

print(f'articles in repo: {len(df_repo)} -> non-english language articles removed')

language
en     437737
es        585
it        304
de        291
fr        277
hr        254
pt        182
ceb       174
pl        134
sv         84
ca         58
nl         56
br         35
la         26
no         21
fi         19
id         19
eo         14
hu          9
eu          6
vi          6
tr          5
ru          4
kn          4
zh          4
gl          3
cy          3
sl          3
war         3
fy          2
uk          2
ro          2
oc          2
ja          2
nn          2
fa          2
lt          2
als         1
bn          1
si          1
hi          1
bar         1
lb          1
nah         1
sr          1
cs          1
et          1
Name: count, dtype: int64
articles in repo: 437737 -> non-english language articles removed


In [6]:
# save to disk
df_repo.to_csv('../data/outputs_pre-processing/repo_eng_pre-processed.csv', encoding='utf=8', index=False) 
df_repo_nas.to_csv("../data/outputs_pre-processing/repo_eng_nas_removed.csv", encoding='utf-8', index=False)
df_repo_dupes.to_csv("../data/outputs_pre-processing/repo_eng_duplicates_removed.csv", encoding='utf-8', index=False)

# read pos data

In [24]:
# ce data 
df_pos = pd.read_csv('../data/from_ce/all_ce_relevant_papers.csv', encoding='utf-8')

# rename columns

df_pos.rename(columns={
    'Title': 'title',
    'Citation': 'citation',
    'Publication_year': 'year'
}, inplace=True)

# normalize title 

df_pos['title'] = [normalize_text(str(title)) for title in df_pos['title']]

# subset metadata

df_pos = df_pos.loc[:,['title','citation','year']] 
print(f'NAs in pos:\n{df_pos.isna().sum()}')
print(f'articles in positives raw: {len(df_pos)}')

# create df of nas and duplicates

df_pos_nas = df_pos[df_pos['title'].isna().isna()] # is zero
df_pos_dupes = df_pos[df_pos.duplicated(subset=['title'], keep=False)]

# drop nas and duplicates

df_pos = df_pos.dropna(subset='title', how='any').drop_duplicates(subset=['title','citation'], keep='first').drop_duplicates(subset=['title'], keep='first')
print(f'articles in positives: {len(df_pos)} -> nas and duplicates removed')

df_pos = df_pos.reset_index()

NAs in pos:
title        0
citation     0
year        15
dtype: int64
articles in positives raw: 15647
articles in positives: 10589 -> nas and duplicates removed


# detect language of titles in pos 



In [26]:
# detect languages

df_pos["language"] = df_pos["title"].apply(get_lang)

# filter english only

df_pos_noneng = df_pos[df_pos['language'] != 'en']
df_pos_eng = df_pos[df_pos['language'] == 'en']
# df_pos = df_pos.reset_index()

print(f'articles in positives: {len(df_pos_eng)} -> english articles')
print(f'articles in positives removed: {len(df_pos_noneng)} -> non-english language articles')
print(df_pos["language"].value_counts())

articles in positives: 9117 -> english articles
articles in positives removed: 1472 -> non-english language articles
language
en     9117
ja      409
de      312
es      135
zh      126
fr      106
ru       81
pt       67
pl       62
hu       52
ko       49
it       34
ar        9
fa        8
nl        7
sv        2
ceb       2
cs        2
ca        2
tr        2
vi        2
uk        1
bn        1
ur        1
Name: count, dtype: int64


In [28]:
# save to disk 
df_pos.to_csv('../data/outputs_pre-processing/pos_eng_pre-processed.csv', encoding='utf-8', index=False) 
df_pos_dupes.to_csv('../data/outputs_pre-processing/pos_eng_duplicates_removed.csv', encoding='utf-8', index=False)
df_pos_noneng.to_csv('../data/outputs_pre-processing/pos_eng_noneng_removed.csv', encoding='utf-8', index=False) 

# remove journals in pos that are not indexed in wos

positive df doesn't have a journal column, but it has a citation column, so we extact the journal name from the citation string

In [29]:
# function to extract journal name from citation
def get_journal_name(citation):
    position1 = citation.rfind('<i>') # finds the position where the pattern appears in text
    position2 = citation.rfind('</i>')
    if position1 == -1: # -1 means error/not found
        return ""
    
    return citation[position1+3:position2]

In [30]:
# get journal names in positive list 

pos_journals = df_pos['citation'].apply(get_journal_name)

# normalize journal names

pos_journals = pos_journals.apply(normalize_text)

# add list of journal names to df_pos

df_pos['journal'] = pos_journals

# get unique journal names

pos_journals = set(pos_journals)
pos_journals.remove("")
print(f'unique journals in positives {len(pos_journals)} >> this includes english and non-english journals')

# remove journals in pos_journals that are not in repo_journals

df_repo_in_pos_journal = df_repo[df_repo['journal'].isin(pos_journals)]
df_repo_in_pos_journal = df_repo_in_pos_journal.drop_duplicates(subset="title")
pos_journals_in_wos = df_repo_in_pos_journal['journal'].unique()

print(f"unique journals in positives that are available in wos {len(pos_journals_in_wos)}")
print(f'articles in repo that are from the journals in pos {len(df_repo_in_pos_journal)}')

# remove journals from df_pos that are not in df_repo

df_pos_in_wos = df_pos[df_pos['journal'].isin(pos_journals_in_wos)]
print(f'articles in pos available in wos {len(df_pos_in_wos)}')

# sorted(pos_journals) 

# read me 
# after inspection I found that journal names that are in blank or are long titles belong to reports that do not follow the journal citation pattern 

unique journals in positives 1183 >> this includes english and non-english journals
unique journals in positives that are available in wos 186
articles in repo that are from the journals in pos 378466
articles in pos available in wos 5474


In [31]:
# save to disk 

df_pos_in_wos.to_csv('../data/outputs_pre-processing/pos_eng_metadta.csv', encoding='utf-8', index=False)
df_repo_in_pos_journal.to_csv('../data/outputs_pre-processing/repo_eng_metadata.csv', encoding='utf-8', index=False)

In [32]:
# data imbalance

len(df_pos_in_wos)/len(df_repo_in_pos_journal)*100

1.4463650631760845